In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage
from typing import TypedDict, Annotated
from langgraph.checkpoint.memory import InMemorySaver
import os
import dotenv

c:\LAE\Learnings\LangGraph\langvenv\Lib\site-packages\langchain_core\_api\deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
dotenv.load_dotenv()

llm = ChatGroq(model="llama-3.3-70b-versatile", api_key=os.getenv("GROQ_API_KEY"))

In [3]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

In [4]:
def generate_joke(state: JokeState):
    response = llm.invoke(f'Tell me a joke about {state["topic"]}')

    return {
        "joke": response.content
    }

In [5]:
def explain_joke(state: JokeState):
    response = llm.invoke(f'Explain why this joke is about {state["joke"]}')
    
    return {
        "explanation": response.content
    }

In [27]:
graph = StateGraph(JokeState)
graph.add_node("generate_joke", generate_joke)
graph.add_node("explain_joke", explain_joke)

graph.add_edge(START, "generate_joke")
graph.add_edge("generate_joke", "explain_joke")
graph.add_edge("explain_joke", END)

checkpointer = InMemorySaver()
app = graph.compile(checkpointer=checkpointer)


In [28]:
config = {
    'configurable': {
        'thread_id': '1'
    }
}

app.invoke({
    'topic': 'cats'
}, config=config)

{'topic': 'cats',
 'joke': 'Why did the cat join a band?\n\nBecause it wanted to be the purr-cussionist!',
 'explanation': 'A classic joke. This joke is funny because it\'s a play on words, using a cat-related pun to create a clever connection between the cat and the band.\n\nThe word "purr" has a double meaning here:\n1. Cats are known for making a "purr" sound, which is a unique vocalization they use to communicate.\n2. "Percussionist" is a musical term that refers to a person who plays percussion instruments, such as drums or cymbals.\n\nBy replacing "per" with "purr", the joke creates a wordplay that links the cat\'s distinctive sound to the musical term, making it a clever and humorous reason for the cat to join a band. It\'s a lighthearted and creative way to use language, which is what makes it a joke!'}

In [29]:
app.get_state(config)

StateSnapshot(values={'topic': 'cats', 'joke': 'Why did the cat join a band?\n\nBecause it wanted to be the purr-cussionist!', 'explanation': 'A classic joke. This joke is funny because it\'s a play on words, using a cat-related pun to create a clever connection between the cat and the band.\n\nThe word "purr" has a double meaning here:\n1. Cats are known for making a "purr" sound, which is a unique vocalization they use to communicate.\n2. "Percussionist" is a musical term that refers to a person who plays percussion instruments, such as drums or cymbals.\n\nBy replacing "per" with "purr", the joke creates a wordplay that links the cat\'s distinctive sound to the musical term, making it a clever and humorous reason for the cat to join a band. It\'s a lighthearted and creative way to use language, which is what makes it a joke!'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10b17c-2b98-6479-8002-a5da7c3f9b4c'}}, metadata={'source': 'loop

In [30]:
list(app.get_state_history(config))

[StateSnapshot(values={'topic': 'cats', 'joke': 'Why did the cat join a band?\n\nBecause it wanted to be the purr-cussionist!', 'explanation': 'A classic joke. This joke is funny because it\'s a play on words, using a cat-related pun to create a clever connection between the cat and the band.\n\nThe word "purr" has a double meaning here:\n1. Cats are known for making a "purr" sound, which is a unique vocalization they use to communicate.\n2. "Percussionist" is a musical term that refers to a person who plays percussion instruments, such as drums or cymbals.\n\nBy replacing "per" with "purr", the joke creates a wordplay that links the cat\'s distinctive sound to the musical term, making it a clever and humorous reason for the cat to join a band. It\'s a lighthearted and creative way to use language, which is what makes it a joke!'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10b17c-2b98-6479-8002-a5da7c3f9b4c'}}, metadata={'source': 'loo

In [10]:
config1 = {
    'configurable': {
        'thread_id': '2'
    }
}

app.invoke({
    'topic': 'Buffalo'
}, config=config1)

{'topic': 'Buffalo',
 'joke': 'Why did the buffalo go to the party in Buffalo?\n\nBecause he heard it was a "herd" of fun and he wanted to "buck" the trend and have a "wild" time in the city! (get it?)',
 'explanation': 'A clever joke. This joke is funny because it\'s a play on words, using multiple puns related to buffalo and parties. Here\'s why it works:\n\n1. "Herd" of fun: The joke starts by saying the buffalo heard it was a "herd" of fun. This is a pun on the word "herd," which refers to a group of buffalo, but also sounds like "heard," as in, the buffalo heard about the party. It\'s a clever connection between the animal and the idea of a social gathering.\n\n2. "Buck" the trend: The next part of the joke says the buffalo wanted to "buck" the trend. This is another pun, as "buck" can refer to a male buffalo (a buck is a male deer or buffalo), but it also means to resist or go against a trend. So, the buffalo is making a joke about being a nonconformist, while also referencing it

In [11]:
app.get_state(config1)

StateSnapshot(values={'topic': 'Buffalo', 'joke': 'Why did the buffalo go to the party in Buffalo?\n\nBecause he heard it was a "herd" of fun and he wanted to "buck" the trend and have a "wild" time in the city! (get it?)', 'explanation': 'A clever joke. This joke is funny because it\'s a play on words, using multiple puns related to buffalo and parties. Here\'s why it works:\n\n1. "Herd" of fun: The joke starts by saying the buffalo heard it was a "herd" of fun. This is a pun on the word "herd," which refers to a group of buffalo, but also sounds like "heard," as in, the buffalo heard about the party. It\'s a clever connection between the animal and the idea of a social gathering.\n\n2. "Buck" the trend: The next part of the joke says the buffalo wanted to "buck" the trend. This is another pun, as "buck" can refer to a male buffalo (a buck is a male deer or buffalo), but it also means to resist or go against a trend. So, the buffalo is making a joke about being a nonconformist, while 

In [12]:
list(app.get_state_history(config1))

[StateSnapshot(values={'topic': 'Buffalo', 'joke': 'Why did the buffalo go to the party in Buffalo?\n\nBecause he heard it was a "herd" of fun and he wanted to "buck" the trend and have a "wild" time in the city! (get it?)', 'explanation': 'A clever joke. This joke is funny because it\'s a play on words, using multiple puns related to buffalo and parties. Here\'s why it works:\n\n1. "Herd" of fun: The joke starts by saying the buffalo heard it was a "herd" of fun. This is a pun on the word "herd," which refers to a group of buffalo, but also sounds like "heard," as in, the buffalo heard about the party. It\'s a clever connection between the animal and the idea of a social gathering.\n\n2. "Buck" the trend: The next part of the joke says the buffalo wanted to "buck" the trend. This is another pun, as "buck" can refer to a male buffalo (a buck is a male deer or buffalo), but it also means to resist or go against a trend. So, the buffalo is making a joke about being a nonconformist, while

### Benefits of Persistence in LangGraph

*   **Memory & Context**: Retain conversation history and state across multiple interactions.
    *   *Example*: A customer support bot remembering a user's previous order ID from a conversation held yesterday.
*   **Fault Tolerance**: Recover and resume graph execution from the last successful checkpoint after a failure.
    *   *Example*: If a server crashes during a complex 10-step data extraction process, the graph resumes from step 6 instead of restarting.
*   **Human-in-the-loop**: Pause execution to wait for human approval or feedback before continuing.
    *   *Example*: An autonomous agent drafts a financial report but pauses for a manager to approve the "Publish" action.
*   **Time Travel**: Inspect, replay, and branch from previous states to debug or test alternative paths.
    *   *Example*: A developer notices an error in step 5, rewinds the state to step 4, modifies the input, and re-runs to see if the fix works.
*   **Long-running Workflows**: Support processes that span long periods of time.
    *   *Example*: An automated hiring assistant that follows up with candidates 3 days after an interview.


### SHORT-TERM MEMORY

In [13]:
"""
Short-Term Memory Chatbot using LangGraph Persistence & Checkpointing

KEY CONCEPTS:
─────────────
1. **Persistence** = Saving the graph's state so it survives across multiple
   `.invoke()` calls. Without it, every call starts from a blank slate.

2. **Checkpoint** = A snapshot of the graph's state at a point in time.
   LangGraph auto-saves one after each node runs.

3. **InMemorySaver** = A simple checkpointer that stores checkpoints in RAM.
   (For production, you'd use SqliteSaver or PostgresSaver.)

4. **thread_id** = Identifies a conversation. Same thread_id = same memory.
   Different thread_id = different conversation history.

HOW IT WORKS:
─────────────
  User msg ──► [chatbot node] ──► LLM processes ALL past messages + new one
                                   ──► Response appended to message list
                                   ──► Checkpoint saved automatically

  Next call with same thread_id:
    Checkpoint restored ──► Old messages are back ──► LLM sees full history
"""

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from typing import TypedDict, Annotated
import os
import dotenv

# ──────────────────────────────────────────────
# 1. Load environment variables
# ──────────────────────────────────────────────
dotenv.load_dotenv()

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=os.getenv("GROQ_API_KEY"),
)

# ──────────────────────────────────────────────
# 2. Define the State
# ──────────────────────────────────────────────
# `add_messages` is a REDUCER — instead of replacing the list,
# it APPENDS new messages to the existing list.
# This is what gives the chatbot "memory" across turns.

class ChatState(TypedDict):
    messages: Annotated[list, add_messages]


# ──────────────────────────────────────────────
# 3. Define the Chatbot Node
# ──────────────────────────────────────────────
# The node receives the FULL message history (restored from checkpoint)
# and sends it all to the LLM so it has context of the conversation.

def chatbot(state: ChatState):
    response = llm.invoke(state["messages"])
    # Return the AI response — `add_messages` will append it to the list
    return {"messages": [response]}

# ──────────────────────────────────────────────
# 4. Build the Graph
# ──────────────────────────────────────────────
graph = StateGraph(ChatState)
graph.add_node("chatbot", chatbot)

graph.add_edge(START, "chatbot")
graph.add_edge("chatbot", END)

# ──────────────────────────────────────────────
# 5. Compile with a Checkpointer (THIS IS THE KEY!)
# ──────────────────────────────────────────────
# Without the checkpointer, the graph forgets everything after each invoke.
# With it, messages persist across calls using the same thread_id.

checkpointer = InMemorySaver()
app = graph.compile(checkpointer=checkpointer)


# ──────────────────────────────────────────────
# 6. Chat Loop
# ──────────────────────────────────────────────
def main():
    print("=" * 50)
    print("  LangGraph Chatbot with Short-Term Memory")
    print("  Type 'quit' to exit, 'new' for new thread")
    print("=" * 50)

    thread_id = "1"  # Same thread_id = same conversation memory

    while True:
        user_input = input(f"\n[Thread {thread_id}] You: ").strip()

        if user_input.lower() == "quit":
            print("Goodbye!")
            break

        # Typing 'new' starts a fresh conversation (new thread_id)
        if user_input.lower() == "new":
            thread_id = str(int(thread_id) + 1)
            print(f"\n--- Started new conversation (Thread {thread_id}) ---")
            print("(Previous thread's memory is still saved separately!)")
            continue

        # The config ties this invocation to a specific thread
        config = {"configurable": {"thread_id": thread_id}}

        # invoke() does:
        #   1. Loads the checkpoint for this thread_id (restores old messages)
        #   2. Appends the new HumanMessage via add_messages reducer
        #   3. Runs the chatbot node (LLM sees full history)
        #   4. Saves a new checkpoint with the updated messages
        result = app.invoke(
            {"messages": [HumanMessage(content=user_input)]},
            config=config,
        )

        # The last message in the list is the AI's response
        ai_response = result["messages"][-1].content
        print(f"\nBot: {ai_response}")


if __name__ == "__main__":
    main()


  LangGraph Chatbot with Short-Term Memory
  Type 'quit' to exit, 'new' for new thread
Goodbye!


### FAULT TOLERENCE

In [14]:
# Fault Tolerence
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [15]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [16]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [17]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [18]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
✅ Step 3 executed


In [19]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)


🔁 Re-running the graph to demonstrate fault tolerance...

✅ Final State: {'input': 'start', 'step1': 'done', 'step2': 'done'}


In [20]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

[StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f10b175-1acd-6189-8003-9bceccded302'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2026-02-16T09:10:20.762958+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f10b175-1aca-6d4f-8002-bde8a1b2dd6d'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=('step_3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f10b175-1aca-6d4f-8002-bde8a1b2dd6d'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-02-16T09:10:20.762024+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f10b175-1140-60e6-8001-d5f6064cb9c7'}}, tasks=(PregelTask(id='c29d9326-e30c-b071-3ae9-51a31481a63

### TIME TRAVEL


In [33]:
app.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f10b17c-2452-621e-8000-05ea5a4d3fdd"}})

StateSnapshot(values={'topic': 'cats'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f10b17c-2452-621e-8000-05ea5a4d3fdd'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2026-02-16T09:13:29.665977+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10b17c-244c-6df0-bfff-1482288d95a7'}}, tasks=(PregelTask(id='6bca3d3e-a477-697d-2cf7-ba491eec5532', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the cat join a band?\n\nBecause it wanted to be the purr-cussionist!'}),), interrupts=())

In [35]:
app.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f10b17c-2452-621e-8000-05ea5a4d3fdd"}})

{'topic': 'cats',
 'joke': 'Why did the cat join a band?\n\nBecause it wanted to be the purr-cussionist!',
 'explanation': 'A classic joke. This joke is about why a cat joined a band because it uses a play on words to create a pun. The word "purr" has a double meaning here:\n\n1. Cats are known for making a "purr" sound, which is a characteristic vocalization they use to communicate and express contentment.\n2. In music, a "percussionist" is a person who plays percussion instruments, such as drums or cymbals.\n\nBy combining these two meanings, the joke creates a pun: "purr-cussionist" sounds similar to "percussionist", but incorporates the cat\'s distinctive sound, "purr". This wordplay is the source of the joke\'s humor, creating a clever and amusing connection between the cat\'s nature and its role in the band.'}

In [36]:
list(app.get_state_history(config))

[StateSnapshot(values={'topic': 'cats', 'joke': 'Why did the cat join a band?\n\nBecause it wanted to be the purr-cussionist!', 'explanation': 'A classic joke. This joke is about why a cat joined a band because it uses a play on words to create a pun. The word "purr" has a double meaning here:\n\n1. Cats are known for making a "purr" sound, which is a characteristic vocalization they use to communicate and express contentment.\n2. In music, a "percussionist" is a person who plays percussion instruments, such as drums or cymbals.\n\nBy combining these two meanings, the joke creates a pun: "purr-cussionist" sounds similar to "percussionist", but incorporates the cat\'s distinctive sound, "purr". This wordplay is the source of the joke\'s humor, creating a clever and amusing connection between the cat\'s nature and its role in the band.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10b17f-98a8-634c-8002-52d921244f85'}}, metadata={'source':

### UPDATING STATE

In [37]:
app.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f10b17c-2452-621e-8000-05ea5a4d3fdd", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f10b182-0bce-6fbd-8001-eb32d062c767'}}

In [39]:
list(app.get_state_history(config))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10b182-0bce-6fbd-8001-eb32d062c767'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2026-02-16T09:16:08.156960+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10b17c-2452-621e-8000-05ea5a4d3fdd'}}, tasks=(PregelTask(id='eda879ff-9631-95b2-89d5-c796f5a4a6f0', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'cats', 'joke': 'Why did the cat join a band?\n\nBecause it wanted to be the purr-cussionist!', 'explanation': 'A classic joke. This joke is about why a cat joined a band because it uses a play on words to create a pun. The word "purr" has a double meaning here:\n\n1. Cats are known for making a "purr" sound, which is a characteristic vocalizatio

In [40]:
app.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f10b182-0bce-6fbd-8001-eb32d062c767"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa go to therapy?\n\nBecause it was feeling a little "crusty" and had a lot of "filling" emotional issues to work through! (get it?)',
 'explanation': 'A deliciously clever joke. Here\'s why it\'s funny:\n\nThe joke relies on a play on words, using puns related to the characteristics of a samosa, a type of fried or baked pastry filled with spices and other ingredients.\n\n1. "Crusty" has a double meaning:\n\t* A samosa has a crust, which is the outer layer of the pastry.\n\t* "Feeling crusty" is an idiomatic expression that means being irritable, grumpy, or having a rough exterior. In this context, the samosa is feeling emotionally rough, much like a person might feel when they\'re struggling with their emotions.\n2. "Filling" emotional issues is another clever pun:\n\t* A samosa has a filling, which refers to the ingredients inside the pastry.\n\t* "Filling emotional issues" is a play on words, implying that the samosa has a lot of emotion

In [41]:
list(app.get_state_history(config))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa go to therapy?\n\nBecause it was feeling a little "crusty" and had a lot of "filling" emotional issues to work through! (get it?)', 'explanation': 'A deliciously clever joke. Here\'s why it\'s funny:\n\nThe joke relies on a play on words, using puns related to the characteristics of a samosa, a type of fried or baked pastry filled with spices and other ingredients.\n\n1. "Crusty" has a double meaning:\n\t* A samosa has a crust, which is the outer layer of the pastry.\n\t* "Feeling crusty" is an idiomatic expression that means being irritable, grumpy, or having a rough exterior. In this context, the samosa is feeling emotionally rough, much like a person might feel when they\'re struggling with their emotions.\n2. "Filling" emotional issues is another clever pun:\n\t* A samosa has a filling, which refers to the ingredients inside the pastry.\n\t* "Filling emotional issues" is a play on words, implying that the samosa 